In [ ]:

!pip install transformers datasets accelerate torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install peft

In [ ]:
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForCausalLM
from datasets import Dataset
import torch
from peft import LoraConfig, get_peft_model

In [ ]:
model_name="meta-llama/Llama-3.2-1B-Instruct"
# model = LlamaForCausalLM.from_pretrained(model_name)
# tokenizer = LlamaTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:

# data = [
#     {"document": "The capital of France is Paris.", "question": "What is the capital of France?", "answer": "Paris"},
#     {"document": "The Eiffel Tower is in Paris.", "question": "Where is the Eiffel Tower?", "answer": "Paris"},

# ]
data = [
    {"document": "The capital of France, Paris, is one of the most popular tourist destinations in the world. Known for its historical landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral, Paris attracts millions of visitors each year. The city is also famous for its culture, fashion, and cuisine, making it a global hub for art, education, and commerce. Paris is often referred to as 'The City of Light' due to its significant role in the Age of Enlightenment and its early adoption of street lighting.", "question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"document": "In Bangladesh, seasonal changes have a profound impact on the economy, especially on sectors such as agriculture, energy, retail, and tourism. These seasonal shifts affect agricultural production cycles, energy demand, and consumer behavior. Understanding the effects of these changes can help businesses and policymakers prepare for economic fluctuations. Bangladesh experiences a tropical monsoon climate, which means the country has distinct seasons that shape its economy. The monsoon season from June to October is critical for rice production, but floods can disrupt the harvest. The dry season from November to February is essential for the Rabi crops like wheat, while the pre-monsoon season is the time for preparing the fields and harvesting early crops.", "question": "What is the impact of seasonal changes on Bangladesh's economy?", "answer": "Seasonal changes in Bangladesh significantly affect agriculture, energy demand, retail behavior, and tourism, with the monsoon season being crucial for rice production and the dry season for Rabi crops."},
    {"document": "The human brain is composed of approximately 86 billion neurons. These neurons are connected by synapses, which are responsible for transmitting electrical signals that allow communication between different parts of the brain. Neurons are classified into three types: sensory neurons, motor neurons, and interneurons. The brain is responsible for controlling nearly all bodily functions, from breathing and heartbeat to complex cognitive functions like memory, learning, and decision-making. The brain's complexity is further enhanced by the neuroplasticity it exhibits, allowing it to reorganize and form new neural connections throughout a person’s life.", "question": "How many neurons are in the human brain?", "answer": "The human brain contains approximately 86 billion neurons."},
    {"document": "The Eiffel Tower, located in Paris, France, was designed by the engineer Gustave Eiffel for the 1889 World's Fair to commemorate the centennial of the French Revolution. It stands at 324 meters (1,063 feet) tall and was the tallest man-made structure in the world until the completion of the Chrysler Building in New York City in 1930. The Eiffel Tower is made of wrought iron and weighs approximately 10,100 tons. It is visited by over 7 million people annually, making it one of the most recognized and visited monuments in the world. The tower is also used for various purposes, including radio transmission.", "question": "When was the Eiffel Tower constructed and who designed it?", "answer": "The Eiffel Tower was constructed between 1887 and 1889 and was designed by Gustave Eiffel."},
    {"document": "Python is a high-level, interpreted programming language known for its easy-to-read syntax and versatility. Created by Guido van Rossum and first released in 1991, Python emphasizes code readability, allowing programmers to write clear and logical code. It supports multiple programming paradigms, including procedural, object-oriented, and functional programming. Python is widely used in various fields, such as web development, data analysis, artificial intelligence, machine learning, and automation. Its vast collection of libraries and frameworks, like Django for web development and TensorFlow for machine learning, makes it an essential tool for modern developers.", "question": "What are the main features of Python?", "answer": "Python is known for its readable syntax, versatility, and support for multiple programming paradigms, and it is widely used in fields like web development, data analysis, and machine learning."},
    {"document": "The Great Wall of China is one of the most famous and impressive structures in the world. It stretches over 13,000 miles, making it the longest wall in the world. The wall was built over several dynasties, starting in the 7th century BC, primarily to protect Chinese states and empires from invasions by various nomadic tribes. The wall was constructed using a variety of materials, including earth, wood, and brick, depending on the region. Today, sections of the Great Wall are popular tourist attractions, and it has been designated as a UNESCO World Heritage site. It symbolizes China’s historical strength and resilience.", "question": "Why was the Great Wall of China built and what materials were used?", "answer": "The Great Wall of China was built to protect against invasions, using materials like earth, wood, and brick depending on the region."},
    {"document": "The Amazon rainforest, often referred to as the 'lungs of the Earth,' is the largest tropical rainforest in the world, covering over 5.5 million square kilometers. It is home to more than 10% of the world’s known species, including plants, animals, and insects. The Amazon is vital to the global climate because it absorbs vast amounts of carbon dioxide and releases oxygen, thus helping to regulate the Earth’s atmosphere. However, the forest is under threat from deforestation, largely due to logging, agriculture, and mining. Efforts to preserve the Amazon have become a significant focus of environmental campaigns globally.", "question": "Why is the Amazon rainforest important to the Earth?", "answer": "The Amazon rainforest is vital for regulating the Earth’s climate by absorbing carbon dioxide and releasing oxygen, and it is home to a vast array of species."},
    {"document": "The United States declared its independence from Great Britain on July 4, 1776. This was a result of growing dissatisfaction with British rule, particularly over issues like taxation without representation, the presence of British troops in the colonies, and restrictions on trade. The declaration was signed by representatives of the 13 colonies at the Continental Congress, and it marked the beginning of the formation of the United States as an independent nation. The Declaration of Independence, written primarily by Thomas Jefferson, is celebrated annually on July 4th as Independence Day in the U.S.", "question": "When did the United States declare independence and why?", "answer": "The United States declared independence on July 4, 1776, due to dissatisfaction with British rule, particularly over taxation without representation."},
    {"document": "Water boils at 100°C (212°F) at sea level under standard atmospheric pressure. As altitude increases, the atmospheric pressure decreases, causing the boiling point of water to lower. For example, at an elevation of 2,000 meters (6,561 feet), the boiling point of water is approximately 93.4°C (200.1°F). The decrease in boiling point with altitude is due to the reduction in air pressure, which makes it easier for water molecules to escape into the gas phase. This phenomenon is important for cooking and scientific experiments that take place at higher altitudes.", "question": "At what temperature does water boil at sea level?", "answer": "Water boils at 100°C (212°F) at sea level under standard atmospheric pressure."},
    {"document": "The Internet was first developed in the late 1960s as part of a project called ARPANET, funded by the U.S. Department of Defense. ARPANET was designed to be a robust, decentralized network that could withstand military attacks during the Cold War. The first message sent over ARPANET was 'LO,' as the system crashed before the word 'LOGIN' could be fully typed. Over the next few decades, ARPANET evolved into the modern Internet, connecting millions of people worldwide. Today, the Internet is an integral part of daily life, used for communication, entertainment, business, education, and much more.", "question": "When was the Internet first developed and why?", "answer": "The Internet was first developed in the late 1960s as part of ARPANET, funded by the U.S. Department of Defense to create a robust, decentralized communication network."},
    {"document": "The human digestive system is responsible for breaking down food into nutrients that the body can absorb. It starts with the mouth, where food is chewed and mixed with saliva, and ends with the anus, where waste is excreted. The food moves through the esophagus to the stomach, where it is mixed with gastric juices. The small intestine is the main site for nutrient absorption, while the large intestine absorbs water and forms waste. The digestive system also involves various organs like the liver, pancreas, and gallbladder, which help process and store nutrients.", "question": "What is the role of the human digestive system?", "answer": "The human digestive system breaks down food into nutrients for absorption and eliminates waste through the anus."},
    {"document": "The Sahara Desert is the largest hot desert in the world, covering much of North Africa. It spans over 9 million square kilometers and experiences extreme temperature fluctuations, with daytime temperatures reaching up to 50°C (122°F) and nighttime temperatures dropping significantly. The desert is home to a variety of unique wildlife, such as camels, snakes, and scorpions. Despite the harsh conditions, the Sahara has been inhabited for thousands of years, with ancient cultures like the Egyptians and Berbers living in and around its borders.", "question": "Where is the Sahara Desert located and what is it known for?", "answer": "The Sahara Desert is located in North Africa and is known for its extreme temperatures and unique wildlife."},
    {"document": "The Pyramids of Giza, located on the outskirts of Cairo, Egypt, are one of the Seven Wonders of the Ancient World. The largest pyramid, the Great Pyramid, was built for the Pharaoh Khufu during the Fourth Dynasty of the Old Kingdom of Egypt. These pyramids were originally covered in smooth limestone, making them shine brightly in the sun, and they were constructed as tombs for the pharaohs and their queens. The construction techniques used to build the pyramids are still debated by historians, but they are considered an extraordinary feat of engineering.", "question": "What are the Pyramids of Giza known for?", "answer": "The Pyramids of Giza are known for being one of the Seven Wonders of the Ancient World and for their remarkable engineering as tombs for Egyptian pharaohs."},
    {"document": "The Amazon River, the second longest river in the world after the Nile, flows through the heart of South America, primarily through Brazil. It has a total length of about 4,345 miles (7,062 kilometers) and drains an area of about 7 million square kilometers. The river and its tributaries form the largest river system in the world by discharge volume, carrying more water than the next seven largest rivers combined. The Amazon River plays a critical role in transporting goods and people and is a vital lifeline for the rainforest communities that depend on it.", "question": "Where is the Amazon River located and what is its significance?", "answer": "The Amazon River is located in South America, primarily through Brazil, and is significant for being the largest river system by discharge volume."},
    {"document": "Mount Everest, the highest mountain on Earth, rises to an elevation of 8,848 meters (29,029 feet) above sea level. It is part of the Himalayan mountain range and straddles the border between Nepal and the Tibet Autonomous Region of China. Mount Everest is known for its extreme weather conditions, making it a challenging climb for mountaineers. It has been the site of numerous expeditions, with Sir Edmund Hillary and Tenzing Norgay being the first to summit it in 1953. Mount Everest is also a symbol of human perseverance and the natural world's raw power.", "question": "What is Mount Everest known for?", "answer": "Mount Everest is known for being the highest mountain on Earth and for its extreme climbing challenges."},
    {"document": "The human circulatory system is responsible for transporting blood, nutrients, gases, and wastes throughout the body. It consists of the heart, blood vessels, and blood. The heart pumps oxygen-rich blood from the lungs to the rest of the body and returns oxygen-poor blood back to the lungs. The circulatory system plays a crucial role in maintaining homeostasis by regulating temperature, pH, and fluid balance. The system also helps protect the body by transporting immune cells and removing waste products from cells.", "question": "What is the role of the human circulatory system?", "answer": "The human circulatory system transports blood, nutrients, gases, and waste throughout the body, and helps maintain homeostasis and protect the body."},
    {"document": "The Pacific Ocean is the largest and deepest ocean on Earth, covering more than 63 million square miles. It stretches from the Arctic Ocean in the north to the Southern Ocean in the south, and from the Americas in the east to Asia and Australia in the west. The Pacific is home to diverse ecosystems and marine life, including coral reefs, deep-sea trenches, and vast open waters. It plays a vital role in global weather patterns, including the El Niño and La Niña phenomena, which affect climate and weather around the world.", "question": "What is the Pacific Ocean known for?", "answer": "The Pacific Ocean is known for being the largest and deepest ocean on Earth and for its influence on global weather patterns."},
    {"document": "The moon is Earth's only natural satellite and the fifth largest moon in the Solar System. It orbits Earth at an average distance of about 384,400 kilometers (238,855 miles) and has a diameter of 3,474 kilometers (2,159 miles). The moon's gravitational pull causes ocean tides on Earth and plays a key role in stabilizing the planet's axial tilt, which affects the climate. The moon's surface is covered with craters, mountains, and flat plains called maria. It has no atmosphere, which means it experiences extreme temperature variations between day and night.", "question": "What role does the moon play in Earth's environment?", "answer": "The moon causes ocean tides and helps stabilize Earth's axial tilt, which affects the planet's climate."}
]


In [ ]:
dataset = Dataset.from_list(data)

In [ ]:
dataset

Dataset({
    features: ['document', 'question', 'answer'],
    num_rows: 18
})

In [ ]:

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    task_type="CAUSAL_LM"
)

In [ ]:
model=get_peft_model(model, lora_config)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:

inputs = [f"document: {item['document']} question: {item['question']}" for item in data]
labels = [f"{item['answer']}" for item in data]

In [ ]:
print(inputs)
print(labels)

["document: The capital of France, Paris, is one of the most popular tourist destinations in the world. Known for its historical landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral, Paris attracts millions of visitors each year. The city is also famous for its culture, fashion, and cuisine, making it a global hub for art, education, and commerce. Paris is often referred to as 'The City of Light' due to its significant role in the Age of Enlightenment and its early adoption of street lighting. question: What is the capital of France?", "document: In Bangladesh, seasonal changes have a profound impact on the economy, especially on sectors such as agriculture, energy, retail, and tourism. These seasonal shifts affect agricultural production cycles, energy demand, and consumer behavior. Understanding the effects of these changes can help businesses and policymakers prepare for economic fluctuations. Bangladesh experiences a tropical monsoon climate, which means 

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:

input_encodings = tokenizer(inputs, truncation=True, padding=True, return_tensors="pt")
label_encodings = tokenizer(labels, truncation=True, padding=True, return_tensors="pt")

In [ ]:
print("Input encodings:", input_encodings)
print("Label encodings:", label_encodings)

Input encodings: {'input_ids': tensor([[128000,   6190,     25,  ..., 128009, 128009, 128009],
        [128000,   6190,     25,  ...,     30, 128009, 128009],
        [128000,   6190,     25,  ..., 128009, 128009, 128009],
        ...,
        [128000,   6190,     25,  ..., 128009, 128009, 128009],
        [128000,   6190,     25,  ..., 128009, 128009, 128009],
        [128000,   6190,     25,  ..., 128009, 128009, 128009]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
Label encodings: {'input_ids': tensor([[128000,    791,   6864,    315,   9822,    374,  12366,     13, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 12

In [ ]:
import torch

class QADataset(torch.utils.data.Dataset):
    def __init__(self, input_encodings, label_encodings):
        self.input_encodings = input_encodings
        self.label_encodings = label_encodings

    def __len__(self):
        return len(self.input_encodings.input_ids)

    def __getitem__(self, idx):
        input_ids = self.input_encodings.input_ids[idx]
        labels = self.label_encodings.input_ids[idx]


        labels = labels[1:].tolist()
        labels.append(tokenizer.pad_token_id)


        if len(labels) < len(input_ids):

            labels.extend([tokenizer.pad_token_id] * (len(input_ids) - len(labels)))

        labels = torch.tensor(labels)
        input_ids = torch.tensor(input_ids)

        return {
            'input_ids': input_ids,
            'labels': labels
        }

In [ ]:

train_dataset = QADataset(input_encodings, label_encodings)

In [ ]:
# def preprocess_function(examples):
#     # Concatenate document and question
#     inputs = tokenizer(examples['document'] + " question: " + examples['question'], truncation=True, padding="max_length", max_length=512)

#     # Tokenize the answer and ensure labels are consistent in length
#     labels = tokenizer(examples['answer'], truncation=True, padding="max_length", max_length=512)

#     # Add labels to the input dictionary
#     inputs['labels'] = labels['input_ids']

#     return inputs

In [ ]:
# Apply the preprocessing to the entire dataset
# tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    eval_steps=500,
    learning_rate=5e-5,
    report_to="tensorboard",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

<ipython-input-36-9077bb231679>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input_ids)


Step,Training Loss


TrainOutput(global_step=9, training_loss=13.874599032931858, metrics={'train_runtime': 3.0957, 'train_samples_per_second': 17.443, 'train_steps_per_second': 2.907, 'total_flos': 48598608199680.0, 'train_loss': 13.874599032931858, 'epoch': 3.0})

In [ ]:
# Save the fine-tuned model
# model.save_pretrained("./fine_tuned_llama2_7b")
# tokenizer.save_pretrained("./fine_tuned_llama2_7b")

In [ ]:
input_text = "document:The Nile River flows through northeastern Africa, making it one of the longest rivers in the world., question: Where does the Nile River flow through?"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, return_attention_mask=True)
inputs = {key: value.to(device) for key, value in inputs.items()}

outputs = model.generate(inputs["input_ids"], attention_mask=inputs["attention_mask"], num_beams=5, no_repeat_ngram_size=2)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


document:The Nile River flows through northeastern Africa, making it one of the longest rivers in the world., question: Where does the Nile River flow through?, answer: Northeastern Africa., additional information: The Nile is approximately 6,695 kilometers
